# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from matplotlib.patches import Rectangle
from ipywidgets import interact, widgets
import os
from scipy.interpolate import interp1d, griddata
from scipy.signal import savgol_filter
from scipy.interpolate import make_interp_spline

# Config

In [2]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/Geschwindigkeit/FURWET/"
# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsFURWET/"
font_title=17
font_ax=13
font_leg=11

In [3]:
outpath = config['outpath_figs']
df_T = pd.read_csv(outpath + "spektralanalyse_ergebnisse_T.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")

# Hilfsfunktionen


In [4]:
def plot_polar_subplot(ax, df, title):
    ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
    freqs = [float(col.split('_')[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in freqs]
    angles = np.deg2rad(df['backazimuth'].values)

    for period, col in zip(periods, ratio_cols):
        r = np.full_like(angles, period)
        values = df[col].values
        sc = ax.scatter(angles, r, c=values, cmap='viridis', alpha=0.75, s=10)

    # Polarplot-Einstellungen
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(170)

    # Logarithmische Radialachse
    ax.set_yscale('log')

    # Radius-Ticks (Periode in Sekunden)
    radial_ticks = [0.1, 0.3, 1, 3, 10]
    ax.set_yticks(radial_ticks)
    ax.set_yticklabels([f"{t} s" for t in radial_ticks])

    ax.set_title(title)

    return sc


In [5]:
def plot_component(ax, df, title):
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in frequencies]
    # Plot nach Tiefenbereichen
    for (dmin, dmax), label in zip(depth_bins, depth_labels):
        df_subset = df[(df["depth"] >= dmin) & (df["depth"] < dmax)]
        num_events = len(df_subset)
        if df_subset.empty:
            continue

        mean_ratios = df_subset[ratio_cols].mean()
        sem_ratios = df_subset[ratio_cols].sem()
        legend_label = f"depth: {label} (n = {num_events})"

        if dmin == 0 and dmax == 700:
            ax.plot(periods, mean_ratios.values, linestyle='--', linewidth=2, label=legend_label)
        else:
            ax.plot(periods, mean_ratios.values, linestyle='-', label=legend_label)

        ax.fill_between(periods,
                        mean_ratios.values - sem_ratios.values,
                        mean_ratios.values + sem_ratios.values,
                        alpha=0.2)

    # Gesamtmittelwert über alle Tiefen (ohne SEM)
    overall_mean = df[ratio_cols].mean()
    ax.plot(periods,
            overall_mean.values,
            color='black',
            linestyle=':',
            linewidth=2,
            label='Overall mean (all depths)')

    ax.set_xlabel("Period [s]",fontsize=font_ax)
    ax.set_ylabel("Mean ratio (FUR/WET)",fontsize=font_ax)
    ax.set_title(title,fontsize=font_title)
    ax.grid(True)
    ax.legend()

In [6]:
# Funktion zur Vorbereitung der Daten
def prepare_plot_data(df):
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in frequencies]
    return ratio_cols, periods

def smooth_line(x, y, points=500):
    x = np.array(x)
    y = np.array(y)
    x_new = np.linspace(x.min(), x.max(), points)
    spline = make_interp_spline(x, y, k=4)  # kubische Interpolation
    y_smooth = spline(x_new)
    return x_new, y_smooth

# Funktion zur Erstellung der Interpolationsdaten
def prepare_interpolation_data(df):
    ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
    freqs = [float(col.split('_')[1][:-2]) for col in ratio_cols]

    theta_list = []
    r_list = []
    z_list = []

    for freq, col in zip(freqs, ratio_cols):
        theta = np.deg2rad(df['backazimuth'].values)
        r = np.full_like(theta, freq)
        z = df[col].values

        theta_list.extend(theta)
        r_list.extend(r)
        z_list.extend(z)

    theta_array = np.array(theta_list)
    r_array = np.array(r_list)
    z_array = np.array(z_list)

    # Periodizität sicherstellen
    theta_array_extended = np.concatenate([theta_array, theta_array + 2 * np.pi])
    r_array_extended = np.concatenate([r_array, r_array])
    z_array_extended = np.concatenate([z_array, z_array])

    return theta_array_extended, r_array_extended, z_array_extended, min(freqs), max(freqs)

def plot_component_with_sem(ax, df, color, label):
    # Frequenzen und Perioden extrahieren
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in frequencies]  # Umrechnung in Periode

    # Mittelwert und SEM berechnen
    mean_ratios = df[ratio_cols].mean()
    sem_ratios = df[ratio_cols].sem()
    num_events = len(df)

    # Arrays erstellen
    period_array = np.array(periods)
    mean_array = mean_ratios.values
    sem_array = sem_ratios.values

    # Nach Perioden sortieren
    sort_idx = np.argsort(period_array)
    period_array = period_array[sort_idx]
    mean_array = mean_array[sort_idx]
    sem_array = sem_array[sort_idx]

    # Plot ohne Glättung
    ax.plot(period_array, mean_array, color=color, linewidth=2, label=f"{label} (n = {num_events})")
    ax.fill_between(period_array,
                    mean_array - sem_array,
                    mean_array + sem_array,
                    color=color,
                    alpha=0.3,
                    label=f"{label} ± SEM")


## Verhältnis gegen Frequenz

In [7]:
fig, ax = plt.subplots(figsize=(10, 6))
df_T = pd.read_csv(outpath + "spektralanalyse_ergebnisse_T.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")

# Komponenten plotten
plot_component_with_sem(ax, df_T, color='blue', label='Transversal component')
plot_component_with_sem(ax, df_Z, color='green', label='Vertical component')

# Achsen und Titel
ax.set_xlabel("Periode (s)",fontsize=font_ax)
ax.set_xscale('log')
ax.set_ylabel("Mean ratio (FUR/WET)",fontsize=font_ax)
ax.set_title("Mean ratio against period (± SEM)",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_leg)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "meanratio_freqency_TandZ_SEM_FURWET_periode.png", dpi=300)
plt.close()


# Tiefen


### Verhältnis gegen Frequenz unterteilt in Tiefengruppen

In [8]:
# Konfiguration
depth_bins = [(0, 100), (100, 700)]
depth_labels = ["0–100 km", "100–700 km"]
full_range = (0, 700)
df_T = pd.read_csv(outpath + "spektralanalyse_ergebnisse_T.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")

In [14]:
# Subplots erstellen
fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Komponenten plotten
plot_component(axs[0], df_T, "Transversal component")
plot_component(axs[1], df_Z, "Vertical component")
axs[0].text(0.02, 0.95, "a)", transform=axs[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
axs[1].set_xscale('log')
axs[0].set_xscale('log')
axs[1].text(0.02, 0.95, "b)", transform=axs[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
# Formatierung & Speichern
plt.suptitle("Mean ratio (FUR/WET) per frequency divided by depth range (± SEM)")
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig(outpath + "meanratio_freqency_depths_FURWET_periode.png", dpi=300)

plt.close()

## Regionen

In [10]:
# Regionen definieren
region_bins = [1, 2, 4, 6, 7]
region_labels = ["Europe", "Asia and Philippins", "North- and Southamerica", "Ozeania", "Elsewhere"]
# CSV-Dateien laden

In [15]:
df_T = pd.read_csv(outpath + "spektralanalyse_ergebnisse_T.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
# Plotdaten extrahieren
ratio_cols_T, frequencies_T = prepare_plot_data(df_T)
ratio_cols_Z, frequencies_Z = prepare_plot_data(df_Z)

# Gesamtdurchschnitt (alle Regionen)
mean_ratios_all_T = df_T[ratio_cols_T].mean()
sem_ratios_all_T = df_T[ratio_cols_T].sem()
mean_ratios_all_Z = df_Z[ratio_cols_Z].mean()
sem_ratios_all_Z = df_Z[ratio_cols_Z].sem()
n_all_T = len(df_T)
n_all_Z = len(df_Z)

# Gültige Regionen sammeln (mind. 30 Events in T **oder** Z)
valid_regions = []
for reg, label in zip(region_bins, region_labels):
    n_T = len(df_T[df_T["region"] == reg])
    n_Z = len(df_Z[df_Z["region"] == reg])
    if n_T >= 30 or n_Z >= 30:
        valid_regions.append((reg, label, n_T, n_Z))

# Subplots vorbereiten
num_plots = len(valid_regions)
fig, axes = plt.subplots(nrows=num_plots, ncols=1, figsize=(12, 4 * num_plots), sharex=True)
if num_plots == 1:
    axes = [axes]

panel_labels = list(string.ascii_lowercase)

for idx, (reg, label, n_T, n_Z) in enumerate(valid_regions):
    ax = axes[idx]

    # --- Transversal-Komponente ---
    if n_T >= 30:
        df_region_T = df_T[df_T["region"] == reg]
        mean_T = df_region_T[ratio_cols_T].mean()
        sem_T = df_region_T[ratio_cols_T].sem()

        ax.plot(frequencies_T, mean_T.values, label=f"T (n={n_T})", color='tab:blue')
        ax.fill_between(frequencies_T,
                        mean_T.values - sem_T.values,
                        mean_T.values + sem_T.values,
                        alpha=0.2, color='tab:blue')

        # Globaler Mittelwert T
        ax.plot(frequencies_T, mean_ratios_all_T.values, linestyle='--', color='navy', label=f"T global (n={n_all_T})")
        ax.fill_between(frequencies_T,
                        mean_ratios_all_T.values - sem_ratios_all_T.values,
                        mean_ratios_all_T.values + sem_ratios_all_T.values,
                        alpha=0.15, color='navy')

    # --- Vertikal-Komponente ---
    if n_Z >= 30:
        df_region_Z = df_Z[df_Z["region"] == reg]
        mean_Z = df_region_Z[ratio_cols_Z].mean()
        sem_Z = df_region_Z[ratio_cols_Z].sem()

        ax.plot(frequencies_Z, mean_Z.values, label=f"Z (n={n_Z})", color='tab:orange')
        ax.fill_between(frequencies_Z,
                        mean_Z.values - sem_Z.values,
                        mean_Z.values + sem_Z.values,
                        alpha=0.2, color='tab:orange')

        # Globaler Mittelwert Z
        ax.plot(frequencies_Z, mean_ratios_all_Z.values, linestyle='--', color='darkorange', label=f"Z global (n={n_all_Z})")
        ax.fill_between(frequencies_Z,
                        mean_ratios_all_Z.values - sem_ratios_all_Z.values,
                        mean_ratios_all_Z.values + sem_ratios_all_Z.values,
                        alpha=0.15, color='darkorange')

    ax.grid(True)
    ax.set_ylabel("Mean ratio (FUR/WET)",fontsize=font_ax)
    if idx == num_plots - 1:
        ax.set_xlabel("Period [s]",fontsize=font_ax)
    ax.set_xscale('log')
    ax.legend(fontsize=font_leg)
    ax.set_title(f"{panel_labels[idx]}) {label}", loc='left', fontsize=12, fontweight='bold')

# Gesamt-Titel und Layout
fig.suptitle("Mean ratio per frequency for each region (+/- SEM)", fontsize=font_title)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(config['outpath_figs'] + "meanratio_frequency_regions_FURWET_combined.png", dpi=300)
plt.close()
